#Load models

In [1]:
import cv2
import numpy as np
import pandas as pd

In [2]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [3]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!git checkout -q 57535c5a79791cb76e36fdb64975271354f10251
!pip install -q -e .

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 430, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 430 (delta 150), reused 132 (delta 128), pack-reused 231
Receiving objects: 100% (430/430), 12.86 MiB | 32.59 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━

In [4]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/content
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-rcqasgf0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-rcqasgf0
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=af9f57cbd1a417bec0f0f2cade7bab4ef8fcd32a35c75657c41cd0346147abe7
  Stored in directory: /tmp/pip-ephem-wheel-cache-3v6biwps/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [5]:
!pip uninstall -y supervision
!pip install -q supervision==0.6.0

import supervision as sv
print(sv.__version__)

Found existing installation: supervision 0.4.0
Uninstalling supervision-0.4.0:
  Successfully uninstalled supervision-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
groundingdino 0.1.0 requires supervision==0.4.0, but you have supervision 0.6.0 which is incompatible.
0.6.0


In [6]:
GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [7]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [8]:
GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [9]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content
/content/weights


In [10]:
SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [11]:
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import Model

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/content/GroundingDINO


/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [13]:
SAM_ENCODER_VERSION = "vit_h"

from segment_anything import sam_model_registry, SamPredictor

sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)

Depth est

In [14]:
from transformers import pipeline
from PIL import Image
pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-large-hf", device=0)

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

#Functions

In [15]:
from segment_anything import SamPredictor
import gc

In [16]:
def get_video_properties(video_path):
    """
    Retrieves video properties such as total frames, frame height, and frame width.

    :param video_path: Path to the video file.
    :return: A tuple containing the total number of frames, frame height, and frame width.
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video file: {}".format(video_path))

    # Get total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get frame height and width
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Release the video capture object
    cap.release()

    return total_frames, frame_height, frame_width, fps

def extract_key_frames(dataframe, threshold, video_path, n_sec_side=1):
    """
    Divide video into 1-second segments and extract all frames from the segment containing at least one key frame
    and an additional n_sec_side seconds (segments) on each side of such a segment. The output is a dictionary
    where each key represents the index of a segment group and the value is a list of frame IDs within that group.

    :param dataframe: DataFrame containing the frame data of potential keyframes.
    :param threshold: The frame_c_score threshold to identify key frames.
    :param video_path: Path to the video file for frame count extraction.
    :param n_sec_side: Number of seconds (segments) to include on each side of a segment containing at least one key frame.
    :return: A dictionary of unique frame IDs from segments closely surrounding those containing at least one key frame,
             grouped by segment group index.
    """

    total_frames, _, _, fps = get_video_properties(video_path)
    segment_duration = 1  # Duration of each segment in seconds
    frames_per_segment = fps * segment_duration

    qualifying_segments = {}

    # Iterate over segments to identify qualifying segments
    for segment_start_frame in range(0, total_frames, frames_per_segment):
        segment_end_frame = min(segment_start_frame + frames_per_segment, total_frames)

        # Check if the segment contains at least one key frame
        if not dataframe[(dataframe['frame_id'] >= segment_start_frame) &
                         (dataframe['frame_id'] < segment_end_frame) &
                         (dataframe['frame_c_score'] >= threshold)].empty:

            # Calculate the extended start and end frame indices for the group
            extended_start_frame = max(0, segment_start_frame - (frames_per_segment * n_sec_side))
            extended_end_frame = min(total_frames, segment_end_frame + (frames_per_segment * n_sec_side))

            # Calculate the group index based on the current segment start frame
            group_index = segment_start_frame // frames_per_segment

            # Adjust group index for the surrounding segments
            group_index = group_index - n_sec_side if (group_index - n_sec_side) > 0 else 0

            # Add all frames in this extended range to the corresponding group in the dictionary
            if group_index not in qualifying_segments:
                qualifying_segments[group_index] = list(range(extended_start_frame, extended_end_frame))
            else:
                # Update the group if it already exists to ensure we're not duplicating segments
                new_frames = set(qualifying_segments[group_index])
                new_frames.update(range(extended_start_frame, extended_end_frame))
                qualifying_segments[group_index] = sorted(new_frames)

    # Ensure the frame IDs in each group are unique and sorted
    for key in qualifying_segments:
        qualifying_segments[key] = sorted(set(qualifying_segments[key]))

    return qualifying_segments

def consolidate_frame_ids(segment_groups):
    """
    Consolidate all frame IDs from grouped segments into a single, sorted list with duplicates removed.

    :param segment_groups: A dictionary where each key is a group index and each value is a list of frame IDs.
    :return: A sorted list of unique frame IDs across all groups.
    """

    all_frames = set()

    for frame_list in segment_groups.values():
        all_frames.update(frame_list)

    # Convert the set to a sorted list
    consolidated_frames = sorted(all_frames)

    return consolidated_frames

def read_frame_from_video(video_path: str, frame_number: int):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cap.release()
    return frame

Depth est

In [17]:
def estimate(frame: np.ndarray):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)
    depth = pipe(im_pil)["depth"]
    return np.asarray(depth).astype(np.int32)

In [18]:
# # to be integrated
# def get_depth_from_frames(video_path, frame_list):
#     torch.cuda.empty_cache()
#     gc.collect()
#     # Obtain total frames and frame dimensions from the video
#     _, frame_height, frame_width,_ = get_video_properties(video_path)

#     # Initialize the depth maps array with zeros
#     depth_maps_array = np.zeros((len(frame_list),) + (frame_height, frame_width), dtype=np.int32)

#     for i in range(len(frame_list)):
#         frame_number = frame_list[i]
#         frame = read_frame_from_video(video_path, frame_number)
#         depth = estimate(frame)
#         depth_maps_array[i] = depth.astype(np.int32)
#         torch.cuda.empty_cache()
#         gc.collect()
#     return depth_maps_array

def get_z_value(depth: np.ndarray, x: int, y: int):
    if y < depth.shape[0] and x < depth.shape[1]:
        return depth[y, x]
    else:
        return "Coordinate out of bounds"

Det and Seg

In [19]:
def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

def det_and_seg(caption: str, image: np.ndarray, box_threshold: float, text_threshold: float, class_idx: int):
    detections, labels = grounding_dino_model.predict_with_caption(
        image=image,
        caption=caption,
        box_threshold=box_threshold,
        text_threshold=text_threshold
    )
    detections.class_id = [class_idx]*(detections.confidence.shape[0])

    detections.mask = segment(
        sam_predictor=sam_predictor,
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
        xyxy=detections.xyxy
    )
    return detections, labels

def initialize_masks_array(total_frames, number_of_classes, frame_height, frame_width):
    # Initialize the array with zeros, indicating no detections
    return np.zeros((total_frames, number_of_classes, frame_height, frame_width), dtype=int)

# # to be integrated
# def get_masks_from_frames(classes, video_path, keyframes_ids, box_threshold, text_threshold):
#     _, frame_height, frame_width,_ = get_video_properties(video_path)

#     # Initialize the masks array
#     masks_array = initialize_masks_array(len(keyframes_ids), len(classes), frame_height, frame_width)

#     for i in range(len(keyframes_ids)):
#         frame_id = keyframes_ids[i]
#         frame = read_frame_from_video(video_path, frame_id)
#         for c, caption in enumerate(classes):
#             detections_c, _ = det_and_seg(
#                 caption=caption,
#                 image=frame,
#                 box_threshold=box_threshold,
#                 text_threshold=text_threshold,
#                 class_idx=c
#             )

#             merged_mask = np.zeros((frame_height, frame_width), dtype=int)
#             for m in range(len(detections_c.mask)):
#                 mask = detections_c.mask[m]
#                 merged_mask += mask.astype(int) * 2**m

#             merged_mask[0][0] = len(detections_c.mask) # record the num of masks at the corner of the array for later extraction ...
#             masks_array[i, c] = merged_mask
#             # torch.cuda.empty_cache()
#             # gc.collect()
#     return masks_array

integrate and get interactions

In [20]:
def normalize_xy(coordinates, width, height):
    normalized_coordinates = []
    for x, y, z in coordinates:
        normalized_x = x / width
        normalized_y = y / height
        normalized_coordinates.append([normalized_x, normalized_y, z])
    return np.array(normalized_coordinates)

# interaction related
def calculate_distance_to_object(joint, object_points):
    joint = joint.reshape(1, 3)
    diff = object_points - joint
    distances = np.linalg.norm(diff, axis=1)
    return distances

def find_interaction_points(joint, object_points, threshold_inter, threshold_near):
    distances = calculate_distance_to_object(joint, object_points)
    interacting_mask = distances < threshold_inter
    near_mask = distances < threshold_near

    is_near = np.any(near_mask)  # Check if any object point is within the 'near' threshold

    if np.any(interacting_mask):
        interacting_points = object_points[interacting_mask, :]
        interacting_distances = distances[interacting_mask]
        # Stack the coordinates and distances into a single array
        interaction = np.hstack((interacting_points, interacting_distances.reshape(-1, 1)))
    else:
        interaction = None

    return is_near, interaction

def interaction_joints_masks(joints_arr, objects, threshold_inter, threshold_near):
    nearby = []
    interactions = []
    for obj in objects:
      interaction_data = []
      nearby_data = []
      for joint in joints_arr:
          is_near, interaction = find_interaction_points(joint, obj, threshold_inter, threshold_near)
          nearby_data.append(is_near)
          interaction_data.append(interaction)
      nearby.append(any(nearby_data))
      interactions.append(interaction_data)
    return nearby, interactions

In [21]:
def extract_key_frame(dataframe, frame_id):
# extract the joints coordinates of a certain frame id from 3D_data_ikea_convert.h5
  df = dataframe.copy()
  for i in range(dataframe.shape[0]):
    for j in range(dataframe.shape[1])[1:]:
      df.iloc[i, j] = dataframe.iloc[i, j][frame_id]
  return df

def get_depth_value(depth_map, x, y):
    if int(y) < depth_map.shape[0] and int(x) < depth_map.shape[1]:
        return depth_map[int(y), int(x)]
    else:
        return "Coordinate out of bounds"

def get_mask_points(mask, depth_map):
    points_3d = []
    object_points = np.argwhere(mask)
    for point in object_points:
        z = get_depth_value(depth_map, point[1], point[0])
        point_3d = (point[1], point[0], z)
        points_3d.append(point_3d)
    return np.array(points_3d)

def map_depth(z2, z1_values, z2_values):
    return np.interp(z2, z2_values, z1_values)

def sort_values(z1_values, z2_values):
    ziped_li = zip(z1_values, z2_values)
    sorted_li = sorted(ziped_li)
    z1ks, z2ks = zip(*sorted_li)
    return z1ks, z2ks

def map_depth_value(dataframe, depth_map):
  # Map z values using numpy linear interpolation
  df = dataframe.copy()
  all_key_z2s = df['z_frame'].tolist()
  # print(all_key_z2s)
  z1ks = []
  z2ks = []
  for j in range(dataframe.shape[0]):
    if float(dataframe.loc[j, 'visibility']) >= 0.9:
      x = int(dataframe.loc[j, 'x_frame'])
      y = int(dataframe.loc[j, 'y_frame'])
      if x>0 and y>0 :
        # print(j)
        z2 = float(dataframe.loc[j, 'z_frame'])
        z1 = get_depth_value(depth_map, x, y)
        z1ks.append(z1)
        z2ks.append(z2)
  z1ks, z2ks = sort_values(z1ks, z2ks)
  mapped_z1s = []
  # sim_coor = ['sim_3D']
  for z2 in all_key_z2s:
      z1_mapped = map_depth(z2, z1ks, z2ks) #mapping_function(z2)
      mapped_z1s.append(z1_mapped)
      # sim_coor.append((x,y,z1_mapped))
  # print(mapped_z1s)
  df['z_mapped'] = mapped_z1s
  # df.loc[8] = sim_coor
  return df

def normalize_depth_map(depth_array):
    depth_array_float = depth_array.astype(np.float32)
    min_val = np.min(depth_array_float)
    max_val = np.max(depth_array_float)
    normalized_depth = (depth_array_float - min_val) / (max_val - min_val)
    return normalized_depth

In [22]:
# def unmerge_masks(merged_mask):
#   num_masks = merged_mask[0][0]
#   merged_mask[0][0] = 0
#   masks = []
#   for i in range(num_masks):
#       extracted_mask = (merged_mask & 2**i) > 0
#       masks.append(extracted_mask)
#   return masks

In [23]:
## dataframe that record the reba scores of each frame
dfk = pd.read_csv("/content/drive/MyDrive/Project/Object_tracking/peaks_output_by_frame.csv")
## dataframe that record the coordinate of each joint in each frame
dfj = df = pd.read_hdf('/content/drive/MyDrive/Project/Object_tracking/3D_data_ikea_convert.h5', key='df')
video_path = "/content/drive/MyDrive/Project/Object_tracking/scan_video1.avi"

extracted_frames = extract_key_frames(dfk, threshold=10, video_path=video_path, n_sec_side=1)
key_frames = consolidate_frame_ids(extracted_frames)
print(extracted_frames)
print(len(extracted_frames))
print(key_frames)
print(len(key_frames))

joint_names = dfj['joint'].tolist()
print(joint_names)
print(len(joint_names))
### hands and feet index: left wrist: 3, right wrist: 6, left ankle: 9, right ankle: 12

{1: [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 2: [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124], 12: [300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371,

In [24]:
# parameters for detection
classes = ['table leg','tabletop']
BOX_TRESHOLD = 0.3
TEXT_TRESHOLD = 0.1

# parameters for finding interactions
nearby_threshold = 0.08
interactions_threshold = 0.04

In [25]:
df_to_plot = pd.DataFrame(columns = ['frame_id', 'joints', 'masks', 'is_near', 'interactions'])

class_num = len(classes)
total_frames, frame_height, frame_width, fps = get_video_properties(video_path)

In [26]:
for i in range(len(key_frames)): # iterate frames
    frame_id = key_frames[i]
    df_to_plot.loc[i, 'frame_id'] = frame_id
    frame = read_frame_from_video(video_path, frame_id)

    # get depth map
    depth_i = estimate(frame).astype(np.int32)
    depth_norm = normalize_depth_map(depth_i) # normalized depth map of frame j

    # get normalized joints coordinates
    df_frame = extract_key_frame(dfj, frame_id)
    df_frame = map_depth_value(df_frame, depth_norm)
    joints_list = []
    for joint in df_frame['joint'].tolist():
        joint_co = df_frame[df_frame['joint'] == joint].iloc[0]
        joint_xyz = (joint_co['x_frame']/1920, joint_co['y_frame']/1080, joint_co['z_mapped'])
        joints_list.append(joint_xyz)
    joints_array = np.array(joints_list)
    df_to_plot.loc[i, 'joints'] = joints_array

    # get mask points and interactions
    maskpoints_list=[]
    nearby_list = []
    interactions_list = []
    for c, caption in enumerate(classes):
        detections_c, _ = det_and_seg(
            caption=caption,
            image=frame,
            box_threshold=BOX_TRESHOLD,
            text_threshold=TEXT_TRESHOLD,
            class_idx=c
        )

        classk_maskpoints = []
        for m in range(len(detections_c.mask)):
            mask = detections_c.mask[m]
            mask_points = get_mask_points(mask, depth_norm)
            mask_points = normalize_xy(mask_points, frame_height, frame_width)
            classk_maskpoints.append(mask_points)
        maskpoints_list.append(classk_maskpoints)
        classk_near, classk_interactions = interaction_joints_masks(joints_array, classk_maskpoints, interactions_threshold, nearby_threshold)
        nearby_list.append(classk_near)
        interactions_list.append(classk_interactions)

    df_to_plot.loc[i, 'masks'] = maskpoints_list
    df_to_plot.loc[i, 'is_near'] = nearby_list
    df_to_plot.loc[i, 'interactions'] = interactions_list
    torch.cuda.empty_cache()
    gc.collect()

df_to_plot

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:962: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of 

,frame_id,joints,masks,is_near,interactions
0,25,"[[0.3770833333333333, -0.023148148148148147, 0...","[[[[1.775 0.33125 0.2375 ], [1.77592593 0.33...","[[False], [False, True]]","[[[None, None, None, None, None, None, None, N..."
1,26,"[[0.3770833333333333, -0.024074074074074074, 0...","[[[[1.775 0.33125 0.23333333], [1.7759...","[[False], [False, True]]","[[[None, None, None, None, None, None, None, N..."
2,27,"[[0.3770833333333333, -0.022222222222222223, 0...","[[[[1.775 0.33125 0.23333333], [1.7759...","[[False], [False, True]]","[[[None, None, None, None, None, None, None, N..."
3,28,"[[0.3770833333333333, -0.021296296296296296, 0...","[[[[1.775 0.33125 0.2375 ], [1.77592593 0.33...","[[False], [False, True]]","[[[None, None, None, None, None, None, None, N..."
4,29,"[[0.37916666666666665, -0.028703703703703703, ...","[[[[1.775 0.33125 0.23430963], [1.7759...","[[False], [False, True]]","[[[None, None, None, None, None, None, None, N..."
...,...,...,...,...,...
1095,3270,"[[0.36875, 0.019444444444444445, 0.99173551797...","[[[[1.775 0.33125 0.22727273], [1.7759...","[[False, False, False], [False]]","[[[None, None, None, None, None, None, None, N..."
1096,3271,"[[0.3645833333333333, 0.004629629629629629, 1....","[[[[1.77407407 0.33125 0.21576764], [1.775 ...","[[False, False, False], [False]]","[[[None, None, None, None, None, None, None, N..."
1097,3272,"[[0.3578125, 0.000925925925925926, 0.995867788...","[[[[1.77407407 0.33125 0.21487603], [1.775 ...","[[False, False, False], [False]]","[[[None, None, None, None, None, None, None, N..."
1098,3273,"[[0.35208333333333336, -0.012962962962962963, ...","[[[[1.77407407 0.33125 0.21161826], [1.775 ...","[[False, False, False], [False]]","[[[None, None, None, None, None, None, None, N..."


In [27]:
df_to_plot.to_pickle('/content/drive/MyDrive/Project/Object_tracking/scan1_joints_and_masks_to_plot_updated1.pkl')